In [1]:
import json
import numpy as np

from ast import literal_eval

In [2]:
BURN_IN = 20

In [3]:
with open('data.json', 'r') as f:
    raw_data = {literal_eval(k): v for k, v in json.load(f).items()}
    
for key in raw_data.keys():
    solver, params = key
    timeseries = raw_data[key][1]
            
    if solver == 'CBM':
        for species in ['P', 'RNA']:
            timeseries[species] = np.array(timeseries[species + 'nuc']) + np.array(timeseries[species + 'cyt'])
            del timeseries[species + 'nuc']
            del timeseries[species + 'cyt']
    
    del timeseries['Gf']
    del timeseries['Gb']
    
    for species in timeseries:
        timeseries[species] = np.array(timeseries[species])[:, BURN_IN:]

summary_statistics_raw = {
        'mean': lambda x: np.mean(np.mean(x, axis=-1), axis=-1),
        'std': lambda x: np.mean(np.std(x, axis=-1), axis=-1),
        'min': lambda x: np.mean(np.min(x, axis=-1), axis=-1),
        'max': lambda x: np.mean(np.max(x, axis=-1), axis=-1),
        }

stat_mean = {k + species: np.mean([s(v[1][species]) for v in raw_data.values()])
            for k, s in summary_statistics_raw.items()
            for species in ['P', 'RNA']}

stat_std = {k + species: np.std([s(v[1][species]) for v in raw_data.values()])
            for k, s in summary_statistics_raw.items()
            for species in ['P', 'RNA']}

print(stat_mean)
print(stat_std)

In [4]:
import tsfresh.feature_extraction.feature_calculators as tsfe
summary_statistics_raw = {
        'longest_strike_below_mean': lambda x: np.apply_along_axis(tsfe.longest_strike_below_mean, 0, x).mean(),
        'longest_strike_above_mean': lambda x: np.apply_along_axis(tsfe.longest_strike_above_mean, 0, x).mean(),
        'mean_abs_change': lambda x: np.apply_along_axis(tsfe.mean_abs_change, 0, x).mean(),
        'maximum': lambda x: np.apply_along_axis(tsfe.maximum, 0, x).mean(),
        'minimum': lambda x: np.apply_along_axis(tsfe.minimum, 0, x).mean(),
        'variance': lambda x: np.apply_along_axis(tsfe.variance, 0, x).mean(),
        }

stat_mean = {k + species: np.mean([s(v[1][species]) for v in raw_data.values()])
            for k, s in summary_statistics_raw.items()
            for species in ['P', 'RNA']}

stat_std = {k + species: np.std([s(v[1][species]) for v in raw_data.values()])
            for k, s in summary_statistics_raw.items()
            for species in ['P', 'RNA']}

In [5]:
print("stat_mean =",stat_mean)
print("stat_std =", stat_std)

stat_mean = {'longest_strike_below_meanP': 5.2377011138613865, 'longest_strike_below_meanRNA': 4.793258818069307, 'longest_strike_above_meanP': 5.651931208745875, 'longest_strike_above_meanRNA': 6.038321214933994, 'mean_abs_changeP': 297.93370793552566, 'mean_abs_changeRNA': 17.48370513204594, 'maximumP': 1531.3910891089108, 'maximumRNA': 91.14778645833333, 'minimumP': 411.66921281971946, 'minimumRNA': 24.466506806930692, 'varianceP': 108241.7390247367, 'varianceRNA': 297.064633632257}
stat_std = {'longest_strike_below_meanP': 2.4667828386324206, 'longest_strike_below_meanRNA': 0.7026448914120297, 'longest_strike_above_meanP': 1.399528430327171, 'longest_strike_above_meanRNA': 1.0580060990298705, 'mean_abs_changeP': 212.0608104949187, 'mean_abs_changeRNA': 7.600031842541941, 'maximumP': 722.3805712604964, 'maximumRNA': 4.232712083773925, 'minimumP': 419.6285935725412, 'minimumRNA': 20.689287835317344, 'varianceP': 132810.87225382286, 'varianceRNA': 253.89911336323922}
